In [1]:
using PyCall
cma = pyimport("cma")

┌ Info: Precompiling PyCall [438e738f-606a-5dbb-bf0a-cddfbfd45ab0]
└ @ Base loading.jl:1273


PyObject <module 'cma' from '/home/mkoehler/.local/lib/python3.8/site-packages/cma/__init__.py'>

In [2]:
using Distributed
N = 10
addprocs(N)

10-element Array{Int64,1}:
  2
  3
  4
  5
  6
  7
  8
  9
 10
 11

In [5]:
@everywhere using DrWatson
@everywhere @quickactivate :Catalyst

In [6]:
function calibrate(cma,N)
    Init = [0.013235189689493238, 1., 1., 1., 1.] #initial guess
    opts = cma.CMAOptions()
    opts["bounds"] = [[0, 0, 0, 0, 0], [1, 50, 50, 30, 30]] 
    opts["popsize"] = N
    weights = cma.recombination_weights.RecombinationWeights(N)
    opts["CMA_recombination_weights"] = weights
    es = cma.CMAEvolutionStrategy(Init, 0.5, opts) 
    
    input_exp = []
    output_exp = []
    for row in CSV.File(datadir("experiment/SBA15_interp_10.csv"); delim = " ")
        push!(input_exp, row.I)
        push!(output_exp, row.O)
    end
    n = 2
    nels = (5n, convert(Int,2.5n),n)
    S = Vec((0.0,0.0,0.0))
    E = Vec((1.0,0.5,0.2))
    grid = generate_grid(Tetrahedron, nels, S, E)
    faces = union(getfaceset(grid,"top"),getfaceset(grid,"bottom"),getfaceset(grid,"left"),getfaceset(grid,"right"),
                   getfaceset(grid,"front"),getfaceset(grid,"back"))
    addfaceset!(grid, "1", faces)
    
    while isempty(es.stop()) 
        solutions = es.ask() 
        fitness = pmap(function eval(input_args) 
                            try 
                               return Catalyst.solve(input_args[1], input_args[2], input_args[3], 
                                          input_exp, output_exp, progress=false, 
                                          microcomp_type=:nonlinear, 
                                          Q=input_args[4], kₙ=input_args[5], 
                                          calibration=true, micromesh=grid)
                             catch err #in case we loose newton convergence in a microscopic computation 
                                return 1e10
                             end
                        end
                            ,solutions) 
        es.tell(solutions,fitness) 
        es.logger.add() 
        es.disp() 
    end
end

calibrate (generic function with 1 method)

In [ ]:
@time calibrate(cma, N)